# News Recommendation

## Import Library

In [ ]:
import ast
import faiss
import numpy as np
import pandas as pd

import google
from google import genai
from datetime import datetime, timezone
from sklearn.metrics.pairwise import cosine_similarity

from datasets import load_dataset
from huggingface_hub import hf_hub_download

## Load Data

In [ ]:
# LOAD DATA

df_old  = pd.read_csv(r'news-dataset/data.csv')
df      = pd.read_csv(r'news-dataset/labeled_data.csv')

# LOAD DATASET
#dataset = load_dataset(path="SandKing/News-Recommendation",
#                       data_files="labeled_news.csv",
#                       split="train",
#                       streaming=False)

# CONVERT TO PANDAS
#df = dataset.to_pandas()


# LOAD FAISS INDEX  (WE DO NOT PROVIDE FAISS FILE DUE TO BIG MEMORY) . U CAN USE HUGGING FACE CODE BELOW (UNCOMMENT-IT!)
faiss_index = faiss.read_index("news_embeddings.faiss")


# LOAD FAISS 
#FAISS_path = hf_hub_download(repo_id = "SandKing/News-Recommendation", 
#                             filename = 'news_embeddings.faiss', 
#                             repo_type = "dataset")
#faiss_index = faiss.read_index(FAISS_path)

print("FAISS index loaded!")
print("Total vectors:", faiss_index.ntotal)

df

FAISS index loaded!
Total vectors: 32735


,id,source,title,image,url,content,date,created_at,updated_at,summary,topic_id,category,confidence
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698528/depo-pl...,"TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00:00,2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Anggota Komisi VII DPR RI Rofik Hananto menyay...,21,Infrastruktur,0.95
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698522/jokowi-...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00:00,2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Presiden Joko Widodo telah memerintahkan Wakil...,21,Infrastruktur,0.95
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698527/hnw-men...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00:00,2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...,2,Politik,0.95
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698540/tim-dok...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00:00,2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...,21,Infrastruktur,0.95
4,87,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698536/bamsoet...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00:00,2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Ketua MPR RI Bambang Soesatyo telah diangkat s...,58,Olahraga,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32730,63563,kumparan,"5 Hewan Aneh yang Hidup di Palung Mariana, Sal...",https://blue.kumparan.com/image/upload/fl_prog...,https://kumparan.com/kumparansains/5-hewan-ane...,menjadi salah satu lubang terdalam dan tergela...,2023-04-11 20:13:25+00:00,2023-04-11 20:20:10.989219,2023-04-11 20:20:10.989219,Peneliti berhasil mengungkap sedikit demi sedi...,3,Agama,0.95
32731,63564,kumparan,"Biadab! Guru SD di Banyuwangi Cabuli Siswinya,...",https://blue.kumparan.com/image/upload/fl_prog...,https://kumparan.com/kumparannews/biadab-guru-...,Seorang guru SD di Banyuwangi ditangkap oleh p...,2023-04-11 20:05:30+00:00,2023-04-11 20:20:10.989219,2023-04-11 20:20:10.989219,Seorang guru SD di Banyuwangi ditangkap polisi...,0,Hukum,0.95
32732,63565,kumparan,Aji Santoso: Jakmania Perlakukan Persebaya den...,https://blue.kumparan.com/image/upload/fl_prog...,https://kumparan.com/kumparanbola/aji-santoso-...,"Pelatih , Aji Santoso, salut dengan sambutan s...",2023-04-11 19:37:49+00:00,2023-04-11 20:20:10.989219,2023-04-11 20:20:10.989219,"Pelatih Persebaya, Aji Santoso, menyampaikan s...",18,Olahraga,0.95
32733,63566,kumparan,Sopir Audi Cium Kaki Ibu & Istri Hamil Usai Si...,https://blue.kumparan.com/image/upload/fl_prog...,https://kumparan.com/kumparannews/sopir-audi-c...,Suasana haru terjadi usai sidang terdakwa Suge...,2023-04-11 19:24:07+00:00,2023-04-11 20:20:10.989219,2023-04-11 20:20:10.989219,"Sugeng Guruh Gautama Legiman, terdakwa kasus k...",0,Hukum,0.95


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32735 entries, 0 to 32734
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          32735 non-null  int64  
 1   source      32735 non-null  object 
 2   title       32735 non-null  object 
 3   image       32575 non-null  object 
 4   url         32735 non-null  object 
 5   content     32294 non-null  object 
 6   date        32735 non-null  object 
 7   created_at  32735 non-null  object 
 8   updated_at  32735 non-null  object 
 9   summary     32725 non-null  object 
 10  topic_id    32735 non-null  int64  
 11  category    32735 non-null  object 
 12  confidence  32735 non-null  float64
dtypes: float64(1), int64(2), object(10)
memory usage: 3.2+ MB


In [4]:
# CHANGE DATA TYPE

#print(f'Embedding Dimension Before Converted to Float : {len(df['embedding'][0])}')

# CHANGE STR LIST TO LIST OF FLOAT
#df['embedding'] = df['embedding'].map(lambda x : np.fromstring(x.strip('[]'), sep=',', dtype= np.float32))

# CHANGE TO DATETIME
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['updated_at'] = pd.to_datetime(df['updated_at'], errors='coerce')

#print(f'Embedding Dimension After Converted to Float : {len(df['embedding'][0])}')

df[['id', 'source', 'title', 'date']].head(4)

,id,source,title,date
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",2023-03-04 06:18:13+00:00
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,2023-03-04 06:04:38+00:00
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,2023-03-04 06:18:04+00:00
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,2023-03-04 06:44:10+00:00


In [ ]:
# STORE VECTOR EMBEDDING ON FAISS

import faiss

# CONVERT EMBEDDINGS INTO 2D MATRIX
embeddings = np.vstack(df['embedding'].values)

# BUILD FAISS
faiss.normalize_L2(embeddings)
faiss_index = faiss.IndexFlatIP(embeddings.shape[1])  # --> CREATE FAISS INDEX
faiss_index.add(embeddings)   # --> PUSH EMBEDDING INTO FAISS INDEX

# SAVE FAISS
faiss.write_index(faiss_index, "news_embeddings.faiss")

print(f'Created FAISS')

In [ ]:
# SAVE NEW CSV

#df = df.drop(columns = ['embedding'])

#df.to_csv(r'labeled_data.csv', index = False)

## Recommend Similar Articles

In [ ]:
# RECOMMEND SIMILAR ARTICLES    # USE THIS CODE IF U WANT USE FAISS INDEX

def similar_articles(article_id, df, faiss_index, top_k = 10):

    # GET TARGET EMBEDDING
    target_idx = df.index[df["id"] == article_id].tolist()[0]
    
    # RECONSTRUCT EMBEDDING FROM FAISS
    target_embed = faiss_index.reconstruct(target_idx).reshape(1, -1)

    # NORMALIZE EMBEDDING
    faiss.normalize_L2(target_embed)

    # SEARCH ALL SIMILAR EMBEDDING ON FAISS INDEX
    probs, index = faiss_index.search(target_embed, top_k + 1)
    
    # STORE SIMILARITY
    results = df.iloc[index[0]].copy()
    results['similarity'] = probs[0]

    # REMOVE ITEM THAT RECOMMEND ITSELF
    results[results["id"] != article_id].head(top_k)

    return results

In [ ]:
# RECOMMEND SIMILAR ARTICLES    ### UNCOMMENT THIS CODE IF U WANT DO NOT USE FAISS INDEX, AND COMMENT CODE ABOVE THIS CELL

#def similar_articles(article_id, df, top_k = 10):

    # GET TARGET EMBEDDING
#    target_article = df[df['id'] == article_id]
#    target_embed   = target_article['embedding'].iloc[0]

    # COMPARE THE TARGET WITH THE WHOLE NEWS
#    sim_scores = cosine_similarity([target_embed], np.stack(df['embedding']))[0]
    
    # STORE SIMILARITY
#    df_result = df.copy()
#    df_result['similarity'] = sim_scores

    # GET THE MOST SIMILAR NEWS
#    top_similar = df_result[df_result['id'] != article_id].sort_values(by= 'similarity', ascending=False).head(top_k)

#    return top_similar

In [10]:
# RECOMMEND

results = similar_articles(article_id = 83,df = df, vector_index = faiss_index, top_k = 10)

results[['id', 'source', 'title', 'date']]

,id,source,title,date
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",2023-03-04 06:18:13+00:00
22005,44726,tempo,Rentetan Kebakaran Pertamina 2023: Depo Plumpa...,2023-04-02 05:58:27+00:00
85,74,tempo,"Kebakaran di Plumpang, Anggota Komisi VII DPR ...",2023-03-04 04:46:12+00:00
4335,9223,cnbcindonesia,"Bos Pertamina Ungkap Kebakaran Depo Plumpang, ...",2023-03-14 10:04:08+00:00
216,292,tempo,"Kebakaran Plumpang, Syarief Hasan Minta Evalua...",2023-03-07 12:06:16+00:00
5,92,tempo,"Korban Tewas Kebakaran Depo Plumpang 17 Orang,...",2023-03-04 09:40:00+00:00
21961,44649,tempo,"Pertamina Tak Putus Dirundung Musibah, 3 Insid...",2023-04-02 03:16:00+00:00
205,282,tempo,Syarief Hasan: Dukacita tragedi Depo Plumpang ...,2023-03-07 12:06:16+00:00
17,91,tempo,Wapres Ma'ruf Amin Usulkan Pemindahan Depo Per...,2023-03-04 08:45:44+00:00
12096,19395,cnnindonesia,DPR Beri Waktu Sebulan ke Bos Pertamina Buka S...,2023-03-16 12:36:33+00:00


## Session-Based Content Recommendation

In [22]:
# RECOMMEND RELEVANT ARTICLES BASED ON USER HISTORY READ

def session_embedding(article_ids : list, df : pd.DataFrame, faiss_index = None, decay_lambda : float = 0.0001, min_weights : float = 0.01):

    # GET THE CURRENT TIME
    now = datetime.now(timezone.utc)
    vectors = []

    # FOR EACH ARTICLES THAT USER READ
    for article_id in article_ids:

        # GET AN ARTICLE THAT USER READ
        article_idx = df.index[df["id"] == article_id].tolist()[0]

        article_date = df.loc[article_idx, "date"]

        # DETERMINE HOW RELEVANT THE ARTICLES NOW 
        delta_days = (now - article_date).days  # CALCULATING THE DIFFERENCE BETWEEN THE CURRENT TIME AND THE TIME THE NEWS WAS PUBLISHED
        weights = np.exp(-decay_lambda * delta_days)     # GIVE LESS WEIGHT TO OLD ARTICLES , AND MORE WEIGHT TO NEW ARTICLES
        weights = max(weights, min_weights)

        # 4. GET EMBEDDING FROM FAISS
        emb = faiss_index.reconstruct(article_idx)

        # COMBINING MANY ARTICLES THAT HAVE BEEN READ INTO 1 NEW EMBEDDING VECTOR
        vectors.append(weights * emb)   # CALCULATE ARTICLE RELEVANCE (BASED ON WEIGHT)
    
    # SUM ALL READ ARTICLES
    new_vectors = np.sum(vectors, axis=0).astype(np.float32).reshape(1, -1)

    # NORMALIZE
    faiss.normalize_L2(new_vectors)

    return new_vectors

In [18]:
# GET TOP-K RELEVANT ARTICLES BASED ON USER HISTORY   ### USE THIS CODE IF U WANT USE FAISS INDEX

def get_relevant_articles(article_ids, df, faiss_index, top_k = 10):

    # AGGREGATE NEW EMBEDDING BASED USER HISTORY
    user_vectors = session_embedding(article_ids, df, faiss_index)

    # SEARCH ALL SIMILAR EMBEDDINGS ON FAISS INDEX
    scores, index = faiss_index.search(user_vectors, top_k + len(article_ids))

    # ADD SIMILARITY FEATURES INTO DATAFRAME
    results = df.iloc[index[0]].copy()
    results["similarity"] = scores[0]

    # REMOVE READ ARTICLES FROM RECOMMENDATION
    results = results[~results["id"].isin(article_ids)]
    top_relevance = results.head(top_k)
    
    return top_relevance

In [ ]:
# GET TOP-K RELEVANT ARTICLES BASED ON USER HISTORY   ### UNCOMMENT THIS CODE IF U WANT DO NOT USE FAISS INDEX, AND COMMENT CODE ABOVE THIS CELL

#def get_relevant_articles(article_ids, df, top_k = 10):

#    # AGGREGATE NEW EMBEDDING BASED USER HISTORY
#    user_vectors = session_embedding(article_ids, df)

#    # SIM SCORE
#    sim_score = cosine_similarity([user_vectors], np.stack(df['embedding'].values))[0]

    # STORE RESULTS
#    df_results = df.copy()
#    df_results['similarity'] = sim_score

    # GET MOST RELEVANCE ARTICLES
#    relevance_article = df_results[~df_results['id'].isin(article_ids)]        # DONT RECOMMEND ARTICLE THAT USER HAVE READ
#    top_relevance = relevance_article.sort_values(by = 'similarity', ascending = False).head(top_k)  

#    return top_relevance

In [23]:
# RECOMEND

results = get_relevant_articles([9496, 90, 84, 26630], df, faiss_index, top_k = 10)

results[['id', 'source', 'title', 'date', 'similarity']]

,id,source,title,date,similarity
27900,55766,kumparan,Korban Pencabulan Oknum Guru Agama di Aceh Uta...,2023-04-07 12:32:40+00:00,0.934366
9901,26588,kumparan,Pelajar di Jakbar Luka Akibat Disabet Penggari...,2023-03-22 08:47:25+00:00,0.933204
11536,62895,tempo,Pengasuh Pesantren di Kabupaten Batang Ditangk...,2023-04-11 07:35:00+00:00,0.932907
13469,20748,kumparan,Kalbar Sepekan: Korupsi BP2TD; Pria di Sungai ...,2023-03-20 02:23:44+00:00,0.932603
22277,45204,tempo,Polisi Tangkap 4 Orang yang Diduga Membakar Fo...,2023-04-02 14:50:00+00:00,0.931739
1396,5088,tempo,Sekelompok Orang Serang Pengendara di Nabire-E...,2023-03-12 14:44:07+00:00,0.930282
53,111,tempo,Top Nasional: Mabes Polri Turun Tangan Usut Pe...,2023-03-05 00:41:08+00:00,0.930069
66,145,tempo,Soal Penyebab Kebakaran Depo Pertamina Plumpan...,2023-03-05 13:29:38+00:00,0.929716
8670,15763,tempo,"Diduga Cabuli 11 Muridnya, Guru Ngaji di Cireb...",2023-03-17 08:10:00+00:00,0.929047
7522,14617,cnnindonesia,4 Anak di Bawah Umur Terlibat Penganiayaan Mau...,2023-03-16 11:09:35+00:00,0.928245


In [ ]:
# RECOMEND

results = get_relevant_articles([9496, 90, 84, 26630], df, faiss_index, top_k = 10)

results[['id', 'source', 'title', 'date', 'similarity']]

,id,source,title,date,similarity
27900,55766,kumparan,Korban Pencabulan Oknum Guru Agama di Aceh Uta...,2023-04-07 12:32:40+00:00,0.934366
9901,26588,kumparan,Pelajar di Jakbar Luka Akibat Disabet Penggari...,2023-03-22 08:47:25+00:00,0.933204
11536,62895,tempo,Pengasuh Pesantren di Kabupaten Batang Ditangk...,2023-04-11 07:35:00+00:00,0.932907
13469,20748,kumparan,Kalbar Sepekan: Korupsi BP2TD; Pria di Sungai ...,2023-03-20 02:23:44+00:00,0.932603
22277,45204,tempo,Polisi Tangkap 4 Orang yang Diduga Membakar Fo...,2023-04-02 14:50:00+00:00,0.931739
1396,5088,tempo,Sekelompok Orang Serang Pengendara di Nabire-E...,2023-03-12 14:44:07+00:00,0.930282
53,111,tempo,Top Nasional: Mabes Polri Turun Tangan Usut Pe...,2023-03-05 00:41:08+00:00,0.930069
66,145,tempo,Soal Penyebab Kebakaran Depo Pertamina Plumpan...,2023-03-05 13:29:38+00:00,0.929716
8670,15763,tempo,"Diduga Cabuli 11 Muridnya, Guru Ngaji di Cireb...",2023-03-17 08:10:00+00:00,0.929047
7522,14617,cnnindonesia,4 Anak di Bawah Umur Terlibat Penganiayaan Mau...,2023-03-16 11:09:35+00:00,0.928245


## Freshness Recommendation

In [25]:
# FUNCTION TO CONTROL THE LATEST RELEVANT NEWS

def freshness_recommendation(df, similarity_weight = 0.5, freshness_weight = 0.5, decay_lambda = 0.001):

    # GET CURRENT DATE
    now = datetime.now(timezone.utc)
    df = df.copy()

    # CALCULATING THE DIFFERENCE BETWEEN THE CURRENT TIME AND THE TIME THE NEWS WAS PUBLISHED  (current date - news published date)
    df['freshness'] = df['date'].apply(lambda d: np.exp(-decay_lambda * (now - d).days))

    # AGGREGATE SIMILARITY WITH FRESHNESS SCORE (SO THAT THE LATEST NEWS IS MORE RELEVANT)
    df['final_score'] = similarity_weight * df['similarity'] + freshness_weight * df['freshness']

    # SORT BY DESCENDING
    df_fresh = df.sort_values(by = 'final_score', ascending = False)

    return df_fresh

## Diversity Sources

In [24]:
# FUNGSI UNTUK MEMBATASI SUMBER ARTIKEL YG SERING MUNCUL
# Limiting frequently appearing news sources (to ensure more diverse news sources)

def source_diversity(df, max_source = 2, top_k = 10):

    selected = []
    source_count = {}

    # FOR EACH SAMPLE
    for _, row in df.iterrows():
        src = row['source']
        
        # IF THE NEWS SOURCE HAS NOT EXCEEDED max_source
        if source_count.get(src, 0) < max_source:
            selected.append(row)
            source_count[src] = source_count.get(src, 0) + 1

        # IF SOURCE HAS EXCEEDED max_source
        if len(selected) >= top_k:
            break

    return pd.DataFrame(selected)

## Final Pipeline

In [26]:
# FINAL PIPELINE (RE-RANKER)

def news_recommender(article_ids, df, faiss_index, top_k = 10):

    # GET RELEVANT NEWS
    top_relevant = get_relevant_articles(article_ids, df, faiss_index, top_k = 100)

    # GET LATEST RELEVANT NEWS (TRADE OFF SIMILARITY VS FRESHNESS)
    reranked = freshness_recommendation(top_relevant, similarity_weight = 0.8, freshness_weight = 0.2)

    # LIMIT SOURCE OF NEWS THAT APPEAR FREQUENTLY
    final_rank = source_diversity(reranked, max_source = 10, top_k = top_k)

    return final_rank[["id", "title", "source", "date", "final_score", 'summary', 'similarity']]

In [27]:
# RECOMEND

results = news_recommender([9496, 90, 84, 26630], df, faiss_index, top_k = 10)

results[['id', 'source', 'title', 'date', 'similarity']]

,id,source,title,date,similarity
27900,55766,kumparan,Korban Pencabulan Oknum Guru Agama di Aceh Uta...,2023-04-07 12:32:40+00:00,0.934366
11536,62895,tempo,Pengasuh Pesantren di Kabupaten Batang Ditangk...,2023-04-11 07:35:00+00:00,0.932907
9901,26588,kumparan,Pelajar di Jakbar Luka Akibat Disabet Penggari...,2023-03-22 08:47:25+00:00,0.933204
22277,45204,tempo,Polisi Tangkap 4 Orang yang Diduga Membakar Fo...,2023-04-02 14:50:00+00:00,0.931739
13469,20748,kumparan,Kalbar Sepekan: Korupsi BP2TD; Pria di Sungai ...,2023-03-20 02:23:44+00:00,0.932603
32731,63564,kumparan,"Biadab! Guru SD di Banyuwangi Cabuli Siswinya,...",2023-04-11 20:05:30+00:00,0.927924
1396,5088,tempo,Sekelompok Orang Serang Pengendara di Nabire-E...,2023-03-12 14:44:07+00:00,0.930282
25654,51217,suara,Dukun Penggandaan Uang Kubur Korban Pasangan L...,2023-04-05 06:15:35+00:00,0.927634
8670,15763,tempo,"Diduga Cabuli 11 Muridnya, Guru Ngaji di Cireb...",2023-03-17 08:10:00+00:00,0.929047
53,111,tempo,Top Nasional: Mabes Polri Turun Tangan Usut Pe...,2023-03-05 00:41:08+00:00,0.930069


# LLM

In [28]:
# ACCESS AND GET GEMINI API KEY

import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

print(f"Connected to Gemini API: {GOOGLE_API_KEY[:5]}***")

Connected to Gemini API: AIzaS***


In [29]:
# ACTIVATE GEMINI API
client = genai.Client(api_key = GOOGLE_API_KEY)

client

## Build Prompt

In [30]:
# BUILD USER CONTEXT (FOR PROMPT)

def build_user_context(article_ids, df, max_chars = 1500):

    # GET ALL ARTICLES THAT USER HAVE READ
    user_articles = df[df['id'].isin(article_ids)]

    # FOR EACH USER ARTICLES
    context = []
    for _, row in user_articles.iterrows():
        text = f"- {row['title']}: {row['summary']}"
        context.append(text)

    full_context = "\n".join(context)
    return full_context[:max_chars]

In [31]:
# TEST FUNCTION
print(build_user_context([9496, 90, 84, 26630], df))

- Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau Lokasi Kebakaran Depo Plumpang: Presiden Joko Widodo telah memerintahkan Wakil Presiden Ma'ruf Amin untuk meninjau langsung lokasi kebakaran depo Pertamina di Plumpang, Jakarta Utara. Kebakaran besar di lokasi tersebut terjadi pada Jumat malam, 3 Maret 2023, dan mengakibatkan belasan warga di sekitar dep
- Isu Pemilu 2024 Ditunda Mencuat Lagi, Ini 5 Tokoh Pewacana Penundaan Pemilu, Luhut Sebut Big Data: Wacana penundaan Pemilu 2024 kembali mencuat. Terbaru, Pengadilan Negeri atau PN Jakarta Pusat mengetok palu Pemilu 2024 ditunda hingga 2025. Usulan penundaan Pemilu 2024 telah disampaikan oleh beberapa pihak, di antaranya PLt Ketum PPP Muhammad Mardiono, Ketum Part
- 9 Anak Jadi Korban Kekerasan Seksual Guru Olahraga di Sijunjung Sumbar: Guru honorer di Kabupaten Sijunjung, Sumatera Barat, dengan inisial AD (45), diamankan polisi setelah diduga melakukan kekerasan seksual terhadap sembilan murid SD. Kekerasan seksual terhadap anak didik it

In [ ]:
# BUILD CANDIDATE CONTEXT

def build_candidate_context(row, max_chars=800):
    text = f"""Judul: {row['title']}
               Ringkasan: {row['summary']}"""
    
    return text.strip()[:max_chars]

In [33]:
def build_explanation_prompt(user_context, 
                             candidate_context):
    prompt = f"""Kamu adalah sistem rekomendasi berita.
                 Riwayat bacaan pengguna: 
                 {user_context}

                 Berita yang direkomendasikan:
                 {candidate_context}
                 
                 Tugasmu:
                 Buat SATU kalimat pendek (maksimal 12 kata) dengan format PERSIS berikut:
                 "Direkomendasikan karena membahas <topik utama>."

                 Aturan keras: - Jangan gunakan kata "relevan", "serupa", atau "pengguna".
                               - Jangan menjelaskan lebih dari satu alasan.
                               - Jangan lebih dari satu kalimat.
                               - Jangan menambahkan konteks lain.
                               - Fokus hanya pada topik inti berita. 
                               Jawaban HARUS satu kalimat pendek."""
    return prompt.strip()

## Generate LLM Explanation

In [49]:
# GENERATE LLM EXPLANATION FOR RELEVANT ARTICLES

def generate_llm_explanation(article_ids, relevant_articles, df):

    # BUILD CONTEXT
    user_context = build_user_context(article_ids, df)
    candidate_context = build_candidate_context(relevant_articles)

    # BUILD PROMPT
    prompt = build_explanation_prompt(user_context, candidate_context)

    print(prompt)

    # LLM GENERATE
    answer = client.models.generate_content(model= "models/gemma-3-27b-it", contents = prompt)

    return answer.text

In [50]:
# NEWS RECOMMENDER WITH LLM EXPLANATION

def news_recommender_explainer(article_ids, df, faiss_index, top_k = 10):

    # GET RECOMMENDATION FIRST (RELEVANT ARTICLES)
    top_articles = news_recommender(article_ids, df, faiss_index, top_k = top_k)

    # FROM TOP ARTICLES, EXPLAIN IT WITH LLM - WHY THOSE ARTICLES ARE RECOMMENDED TO USER
    explanations = []
    for _, row in top_articles.iterrows():

        # EXPLAIN EACH ARTICLE WITH LLM
        explanation = generate_llm_explanation(article_ids = article_ids, relevant_articles = row, df = df)
        explanations.append(explanation)

    top_articles['explanation'] = explanations

    return top_articles

In [52]:
# FINAL RECOMMENDATION

history = [9496, 90, 84, 26630]
results = news_recommender_explainer(article_ids = history, df = df, faiss_index = faiss_index, top_k = 5)

results

Kamu adalah sistem rekomendasi berita.
                 Riwayat bacaan pengguna: 
                 - Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau Lokasi Kebakaran Depo Plumpang: Presiden Joko Widodo telah memerintahkan Wakil Presiden Ma'ruf Amin untuk meninjau langsung lokasi kebakaran depo Pertamina di Plumpang, Jakarta Utara. Kebakaran besar di lokasi tersebut terjadi pada Jumat malam, 3 Maret 2023, dan mengakibatkan belasan warga di sekitar dep
- Isu Pemilu 2024 Ditunda Mencuat Lagi, Ini 5 Tokoh Pewacana Penundaan Pemilu, Luhut Sebut Big Data: Wacana penundaan Pemilu 2024 kembali mencuat. Terbaru, Pengadilan Negeri atau PN Jakarta Pusat mengetok palu Pemilu 2024 ditunda hingga 2025. Usulan penundaan Pemilu 2024 telah disampaikan oleh beberapa pihak, di antaranya PLt Ketum PPP Muhammad Mardiono, Ketum Part
- 9 Anak Jadi Korban Kekerasan Seksual Guru Olahraga di Sijunjung Sumbar: Guru honorer di Kabupaten Sijunjung, Sumatera Barat, dengan inisial AD (45), diamankan polisi setelah didu

,id,title,source,date,final_score,summary,similarity,explanation
27900,55766,Korban Pencabulan Oknum Guru Agama di Aceh Uta...,kumparan,2023-04-07 12:32:40+00:00,0.819611,Seorang oknum ASN guru agama salah satu SD di ...,0.934366,Direkomendasikan karena membahas kasus kekeras...
11536,62895,Pengasuh Pesantren di Kabupaten Batang Ditangk...,tempo,2023-04-11 07:35:00+00:00,0.818734,Gubernur Jawa Tengah Ganjar Pranowo mengutuk a...,0.932907,Direkomendasikan karena membahas kasus kekeras...
9901,26588,Pelajar di Jakbar Luka Akibat Disabet Penggari...,kumparan,2023-03-22 08:47:25+00:00,0.817537,Polisi menangkap 8 pelajar yang menyerang pela...,0.933204,Direkomendasikan karena membahas kasus pengani...
22277,45204,Polisi Tangkap 4 Orang yang Diduga Membakar Fo...,tempo,2023-04-02 14:50:00+00:00,0.817151,Empat pria ditangkap karena diduga melakukan p...,0.931739,Direkomendasikan karena membahas penangkapan t...
13469,20748,Kalbar Sepekan: Korupsi BP2TD; Pria di Sungai ...,kumparan,2023-03-20 02:23:44+00:00,0.816915,"Berita terbaru dari Kalimantan Barat, mulai da...",0.932603,Direkomendasikan karena membahas kasus krimina...


In [76]:
# FINAL RECOMMENDATION

history = [9496, 90, 84, 26630]
results = news_recommender_explainer(article_ids = history, df = df, top_k = 10)

results

,id,title,source,date,final_score,summary,similarity,explanation
27900,55766,Korban Pencabulan Oknum Guru Agama di Aceh Uta...,kumparan,2023-04-07 12:32:40+00:00,0.819684,Seorang oknum ASN guru agama salah satu SD di ...,0.934366,Direkomendasikan karena membahas kasus kekeras...
11536,62895,Pengasuh Pesantren di Kabupaten Batang Ditangk...,tempo,2023-04-11 07:35:00+00:00,0.818734,Gubernur Jawa Tengah Ganjar Pranowo mengutuk a...,0.932907,Direkomendasikan karena membahas kasus kekeras...
9901,26588,Pelajar di Jakbar Luka Akibat Disabet Penggari...,kumparan,2023-03-22 08:47:25+00:00,0.817537,Polisi menangkap 8 pelajar yang menyerang pela...,0.933204,Direkomendasikan karena membahas kasus pengani...
22277,45204,Polisi Tangkap 4 Orang yang Diduga Membakar Fo...,tempo,2023-04-02 14:50:00+00:00,0.817223,Empat pria ditangkap karena diduga melakukan p...,0.931739,Direkomendasikan karena membahas penangkapan t...
13469,20748,Kalbar Sepekan: Korupsi BP2TD; Pria di Sungai ...,kumparan,2023-03-20 02:23:44+00:00,0.816915,"Berita terbaru dari Kalimantan Barat, mulai da...",0.932603,Direkomendasikan karena membahas kasus krimina...
32731,63564,"Biadab! Guru SD di Banyuwangi Cabuli Siswinya,...",kumparan,2023-04-11 20:05:30+00:00,0.814820,Seorang guru SD di Banyuwangi ditangkap polisi...,0.927924,Direkomendasikan karena membahas kekerasan sek...
1396,5088,Sekelompok Orang Serang Pengendara di Nabire-E...,tempo,2023-03-12 14:44:07+00:00,0.814564,Sejumlah orang menyerang aparat kepolisian saa...,0.930282,Direkomendasikan karena membahas tindak kekera...
25654,51217,Dukun Penggandaan Uang Kubur Korban Pasangan L...,suara,2023-04-05 06:15:35+00:00,0.814082,Kapolda Jawa Tengah mengatakan bahwa seluruh k...,0.927634,Direkomendasikan karena membahas kasus pembunu...
8670,15763,"Diduga Cabuli 11 Muridnya, Guru Ngaji di Cireb...",tempo,2023-03-17 08:10:00+00:00,0.813858,Seorang guru ngaji berinisial S ditangkap poli...,0.929047,Direkomendasikan karena membahas kasus kekeras...
53,111,Top Nasional: Mabes Polri Turun Tangan Usut Pe...,tempo,2023-03-05 00:41:08+00:00,0.813833,Mabes Polri turun tangan menyelidiki penyebab ...,0.930069,Direkomendasikan karena membahas kebakaran dep...


In [44]:
results['explanation'].tolist()

['Direkomendasikan karena membahas kasus penculikan anak.',
 'Direkomendasikan karena membahas aksi pencabulan yang dilakukan Wildan.',
 'Direkomendasikan karena membahas aksi penganiayaan pelajar.',
 'Direkomendasikan karena membahas pembakaran foto tokoh publik.',
 'Direkomendasikan karena membahas korupsi BP2TD dan pembunuhan ibu muda.',
 'Direkomendasikan karena membahas kasus pencabulan anak.',
 'Direkomendasikan karena membahas aksi penyerangan polisi di Papua.',
 'Direkomendasikan karena membahas pembunuhan dukun penggandaan uang.',
 'Direkomendasikan karena membahas kasus penganiayaan anak.',
 'Direkomendasikan karena membahas kebakaran Depo Pertamina Plumpang.']

In [42]:
results['explanation'].tolist()

['Direkomendasikan karena membahas kekerasan seksual terhadap anak di lingkungan pendidikan.',
 'Direkomendasikan karena membahas kekerasan seksual terhadap anak dan penyalahgunaan kuasa.',
 'Direkomendasikan karena membahas kasus penganiayaan pelajar dan penangkapan pelaku.',
 'Direkomendasikan karena membahas tindakan kriminal dan penegakan hukum terkait tokoh publik.',
 'Direkomendasikan karena membahas kasus kriminalitas dan korupsi di daerah.',
 'Direkomendasikan karena membahas kekerasan seksual terhadap anak di lingkungan pendidikan.',
 'Direkomendasikan karena membahas tindak kekerasan dan situasi keamanan di Papua.',
 'Direkomendasikan karena membahas kasus kriminalitas dan korban pembunuhan massal.',
 'Direkomendasikan karena membahas kekerasan seksual terhadap anak di lingkungan pendidikan.',
 'Direkomendasikan karena membahas kebakaran depo Pertamina Plumpang dan penundaan Pemilu.']

In [ ]:
results['explanation'].tolist()

['Direkomendasikan karena membahas kekerasan seksual terhadap anak di lingkungan pendidikan.',
 'Direkomendasikan karena membahas kekerasan seksual terhadap anak dan penyalahgunaan kuasa.',
 'Direkomendasikan karena membahas kasus penganiayaan pelajar dan penangkapan pelaku.',
 'Direkomendasikan karena membahas tindakan kriminal dan penegakan hukum terkait tokoh publik.',
 'Direkomendasikan karena membahas kasus kriminalitas dan korban pembunuhan massal.',
 'Direkomendasikan karena membahas kekerasan seksual terhadap anak dan remaja.',
 'Direkomendasikan karena membahas kebakaran Depo Pertamina Plumpang dan proses investigasinya.',
 'Direkomendasikan karena membahas kekerasan terhadap anak dan kasus penganiayaan.',
 'Direkomendasikan karena membahas kekerasan seksual terhadap anak di lingkungan pendidikan.',
 'Direkomendasikan karena membahas kasus kekerasan seksual terhadap anak.']